### 1. From HTML

*Using only beautiful soap, no regex*

Save in a dataframe the next information using web scraping. Each row of the dataframe must have in different columns:

- The name of the title
- The id of the div where is the value scraped. If there is not id, then the value is must be numpy.nan
- The name of the tag where is the value scraped.
- The next scraped values in different rows: 
    - The value: "Este es el segundo párrafo"  --> Row 1
    - The url https://pagina1.xyz/ --> Row 2
    - The url https://pagina4.xyz/ --> Row 3
    - The url https://pagina5.xyz/ --> Row 4
    - The value "links footer-links" --> Row 5
    - The value "Este párrafo está en el footer" --> Row 6

In [25]:
html = """<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Página de prueba</title>
</head>
<body>
<div id="main" class="full-width">
    <h1>El título de la página</h1>
    <p>Este es el primer párrafo</p>
    <p>Este es el segundo párrafo</p>
    <div id="innerDiv">
        <div class="links">
            <a href="https://pagina1.xyz/">Enlace 1</a>
            <a href="https://pagina2.xyz/">Enlace 2</a>
        </div>
        <div class="right">
            <div class="links">
                <a href="https://pagina3.xyz/">Enlace 3</a>
                <a href="https://pagina4.xyz/">Enlace 4</a>
            </div>
        </div>
    </div>
    <div id="footer">
        <!-- El footer -->
        <p>Este párrafo está en el footer</p>
        <div class="links footer-links">
            <a href="https://pagina5.xyz/">Enlace 5</a>
        </div>
    </div>
</div>
</body>
</html>"""

## Sacamos el html ##

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy

def show_html(html_str):
    print(BeautifulSoup(str(html_str), 'html.parser').prettify())


soup = BeautifulSoup(html, 'html.parser')
show_html(soup.text)

Página de prueba



El título de la página
Este es el primer párrafo
Este es el segundo párrafo


Enlace 1
Enlace 2



Enlace 3
Enlace 4





Este párrafo está en el footer

Enlace 5



## LINKS URL ##

In [27]:
#Etiqueta A
links = soup.findAll('a')
links

[<a href="https://pagina1.xyz/">Enlace 1</a>,
 <a href="https://pagina2.xyz/">Enlace 2</a>,
 <a href="https://pagina3.xyz/">Enlace 3</a>,
 <a href="https://pagina4.xyz/">Enlace 4</a>,
 <a href="https://pagina5.xyz/">Enlace 5</a>]

In [28]:
#Etiqueta href

list_of_links = []
acum = 0
for link in links:
    href = link.get('href')
    if href and href[:4] == "http" and acum == 0:
        print(href)
        list_of_links.append(href)

    elif href and href[:4] == "http" and acum == 3:
        print(href)
        list_of_links.append(href)
    
    elif href and href[:4] == "http" and acum == 4:
        print(href)
        list_of_links.append(href)

    acum += 1

list_of_links

https://pagina1.xyz/
https://pagina4.xyz/
https://pagina5.xyz/


['https://pagina1.xyz/', 'https://pagina4.xyz/', 'https://pagina5.xyz/']

## Segundo párrafo y Footer párrafo ##

In [29]:
#Etiqueta p
todos_los_parrafos = soup.findAll('p')
todos_los_parrafos

[<p>Este es el primer párrafo</p>,
 <p>Este es el segundo párrafo</p>,
 <p>Este párrafo está en el footer</p>]

In [30]:
#Cogemos los párrafos segundo y footer
acum = 0
for p in todos_los_parrafos:
    acum +=1
    if acum >1 and acum <3:
        segundo_parrafo = p.contents
    if acum >2 and acum <4:
        parrafo_footer = p.contents

print(segundo_parrafo)
parrafo_footer

['Este es el segundo párrafo']


['Este párrafo está en el footer']

## Class Footer ##

In [31]:
#Etiqueta class
footer_links = soup.find(class_='links footer-links')
show_html(footer_links)

<div class="links footer-links">
 <a href="https://pagina5.xyz/">
  Enlace 5
 </a>
</div>


In [32]:
#Cogemos lo que hay dentro de class
clasecita = footer_links.get('class')
clasecita

['links', 'footer-links']

## Los values ##

In [33]:
print(segundo_parrafo)
print(list_of_links)
print(parrafo_footer)
print(clasecita)

Values = [segundo_parrafo[0], list_of_links[0], list_of_links[1], list_of_links[2], clasecita, parrafo_footer[0]]
Values

['Este es el segundo párrafo']
['https://pagina1.xyz/', 'https://pagina4.xyz/', 'https://pagina5.xyz/']
['Este párrafo está en el footer']
['links', 'footer-links']


['Este es el segundo párrafo',
 'https://pagina1.xyz/',
 'https://pagina4.xyz/',
 'https://pagina5.xyz/',
 ['links', 'footer-links'],
 'Este párrafo está en el footer']

## El título ##

In [34]:
#Etiqueta title

parrafo = soup.find('title')
title_parrafo = parrafo.contents
title_parrafo

#Longitud de title tantas veces como Values haya
title2 = []

for x in range(len(Values)):
    title2.append(title_parrafo)

title2

[['Página de prueba'],
 ['Página de prueba'],
 ['Página de prueba'],
 ['Página de prueba'],
 ['Página de prueba'],
 ['Página de prueba']]

## Las etiquetas ##

In [35]:
tags = ["title", "a", "a", "a", "class", "p"]

## Los id ##

In [36]:
ids = []
for id in soup.select("div[id]"):
    ids.append(id["id"])

print(ids) 

ids_lista = [ids[0], numpy.nan, numpy.nan, numpy.nan, numpy.nan, ids[2]]
ids_lista

['main', 'innerDiv', 'footer']


['main', nan, nan, nan, nan, 'footer']

## El dataframe ##

In [37]:
dict_df = {"Title": title2, "id": ids_lista, "tags" : tags, "Values": Values}

df = pd.DataFrame(dict_df)
df

,Title,id,tags,Values
0,[Página de prueba],main,title,Este es el segundo párrafo
1,[Página de prueba],NaN,a,https://pagina1.xyz/
2,[Página de prueba],NaN,a,https://pagina4.xyz/
3,[Página de prueba],NaN,a,https://pagina5.xyz/
4,[Página de prueba],NaN,class,"[links, footer-links]"
5,[Página de prueba],footer,p,Este párrafo está en el footer


### 2. From Amazon

*Using  beautiful soap  (regex optional)*

Save in a dataframe the next information using web scraping. Using product pages from Amazon, do the following: 

- Get the product name from the web and save it in a column called "item_name"
- Get the price from the web and save it in a column called "item_price"

While you are doing the exercise, document the steps you are doing. Try to do the program for generic pages. If you cannot do it generic, explain the reasons. 

*We recommend to get the source-code, save it in a local file and work from there. It is possible that Amazon detects that you are using webscraping and it changes the source code to avoid possibles attacks.*

-------------------------------

**Example:** 

url = https://www.amazon.es/Tommy-Hilfiger-UM0UM00054-Camiseta-Hombre/dp/B01MYD0T1F/ref=sr_1_1?dchild=1&pf_rd_p=58224bec-cac9-4dd2-a42a-61b1db609c2d&pf_rd_r=VZQ1JTQXFVRZ9E9VSKX4&qid=1595364419&s=apparel&sr=1-1

*item_name* --> "Tommy Hilfiger Logo Camiseta de Cuello Redondo,Perfecta para El Tiempo Libre para Hombre"

*item_price* --> [[18,99 € - 46,59 €]] or one of the options.


